In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.express as px
from jupyter_dash import JupyterDash
import plotly.graph_objects as go
from dash import no_update

In [ ]:
# Create a dash application
app = JupyterDash(__name__)
JupyterDash.infer_jupyter_proxy_config()

app.config.suppress_callback_exceptions = True

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

df_launchsite = []
df_launchsite = list( spacex_df['Launch Site'].unique() )
df_launchsite.append('ALL SITE')

In [ ]:
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                               html.Div([
                                    html.Div(
                                                [
                                                html.H3('Choose Launch Site:', style={'margin-right': '2em'})
                                                ]
                                            ),
                                    dcc.Dropdown(id='input-launchsite', 
                                                 options=[
                                                     {'label': i, 'value': i} for i in df_launchsite],
                                                 placeholder="Select a Launch Site",
                                                 value='ALL SITE',
                                                 searchable=True,
                                                 style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                        ], style={'display': 'flex'}),  
                                html.Br(),

                                
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),                                
                                html.Div( dcc.RangeSlider(id='slider',
                                                min=min_payload,
                                                max=max_payload,
                                                step=100,
                                                value=[min_payload,max_payload],
                                                )
                                        ),
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

@app.callback(
    [   Output(component_id="success-pie-chart", component_property="figure"),
        Output(component_id="success-payload-scatter-chart", component_property="figure")
    ],     
    [   Input(component_id="input-launchsite", component_property="value"),
        Input(component_id="slider", component_property="value")
    ]
)
def generate_chart(selected_site,selected_slider):    
    payload_min =selected_slider[0]
    payload_max =selected_slider[1]    
    
    if selected_site == "ALL SITE":
        fig = px.pie(spacex_df, values='class', names='Launch Site', title='Total Success Launch by Site')
        df1 = spacex_df[ spacex_df['Payload Mass (kg)'].between(payload_min,payload_max)  ]
    else:
        df = spacex_df[ spacex_df['Launch Site'].str.contains(selected_site) ]
        fig = px.pie(df, names='class', title='Total Success/Failure of Selected Site')
        df1 = spacex_df[ spacex_df['Payload Mass (kg)'].between(payload_min,payload_max) & spacex_df['Launch Site'].str.contains(selected_site)  ]
    
    fig1 = px.scatter(df1, x="Payload Mass (kg)", y="class")    
    
    return [fig,fig1]


In [ ]:
# Run the app
if __name__ == '__main__':
    # REVIEW8: Adding dev_tools_ui=False, dev_tools_props_check=False can prevent error appearing before calling callback function
    app.run_server(mode="inline", host="localhost", debug=False, dev_tools_ui=False, dev_tools_props_check=False)